# Assignment 7: Semantic Segmentation
Heute werden wir einfache Netzwerkarchitekturen für "Semantic Segmentation" testen. Ziel ist es dieses Paper in den Grundzügen zu implementieren: https://people.eecs.berkeley.edu/~jonlong/long_shelhamer_fcn.pdf. Bitte lesen!

## Daten

Es gibt einige gute Datensätze, die ihr (bei gegebener Hardware) herunterladen und benutzen könnt. Für diejenigen, die auf CPUs rechnen gilt immer der Tip: Bilder downsamplen!

Sucht Euch einen Satensatz aus: 

KITTI: http://www.cvlibs.net/download.php?file=data_semantics.zip (~300 MB, 200 Bilder)

DUS: http://www.6d-vision.com/scene-labeling (~3 GB, 500 Bilder)

MIT. http://sceneparsing.csail.mit.edu/ (~1 GB, links siehe auf Seite)

## Exc. 7.1 Fully convolutional network, no downsampling
Implementiere die in der Vorlesung besprochene Netzwerkarchitektur von aufeinanderfolgenden CONV-Schichten (stride=1, mit zero-padding), um eine Ausgabeschicht zu bekommen, die die Eingabegröße aufweist. Tip: die letzte CONV-Schicht sollte eine Tiefe haben, die zur Zahl der Klassen korrespondiert. Benutze den L2-Loss zum Labelbild (Achtung: ihr müsst dafür entweder das Labelbild oder den Ausgabetensor umformulieren).

Trainiere das Netzwerk auf den von Dir gewählten Datensatz und zeige den Verlauf des Losses, und einige zufällig gewählte Beispielbilder mit ihren vorhergesagten Segmentierungen an. (**RESULT**)

In [51]:
import os
#os.environ["CUDA_VISIBLE_DEVICES"]="-1"
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session, get_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config))

from keras.preprocessing.image import ImageDataGenerator
#
from keras.models import Sequential
from keras.layers import Dense, Flatten, Convolution2D, MaxPooling2D, Dropout, UpSampling2D
from keras.optimizers import RMSprop
from keras.datasets import mnist
from keras.utils import np_utils
from keras import initializers
from keras import backend as K
from keras.applications import resnet50
from keras.models import load_model
from keras.models import Model
from keras.utils.np_utils import to_categorical


from PIL import Image

In [58]:
print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
print("Hi Tim, vorrangig aus GPU-Gründen haben wir die Jan(n)is-Gruppe bei dieser Übung als auch bei der nächsten mit ins Boot geholt.")
print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Hi Tim, vorrangig aus GPU-Gründen haben wir die Jan(n)is-Gruppe bei dieser Übung als auch bei der nächsten mit ins Boot geholt.
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


In [52]:
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt

image_generator = ImageDataGenerator()
label_generator = ImageDataGenerator()

# Beispiel für den KITTI-Datensatz. Ich habe die 200 training samples in 180 train- und 20 testbilder
# geteilt (macht 180 samples inkl. labels)
# um uns das Leben leichter zu machen, Bilder heruntersamplen
img_size = (40, 128)


# Bild- und Label-Generator
Q1 = image_generator.flow_from_directory( './data_semantics/training/images',
                                        class_mode = None,
                                        batch_size=1,
                                        target_size=img_size, seed=1)

Q2 = label_generator.flow_from_directory( './data_semantics/training/labels',
                                        color_mode = 'grayscale',
                                        class_mode = None,
                                        batch_size=1,
                                        target_size=img_size, seed=1)

# ... kombinieren
train_generator = zip(Q1, Q2)

# mach der Definition des Modells:
# model.fit_generator( train_generator, steps_per_epoch = 1000, epochs = 100)


Found 200 images belonging to 1 classes.
Found 200 images belonging to 1 classes.


In [56]:
prob_drop_conv = 0.2
opt = RMSprop(lr=0.01, rho=0.9)

model = Sequential()
#wir haben zuerst die Struktur gebaut mit Up und Downsampling und das dann nur auskommentiert
#funktioniert leider trotzdem nicht so ganz ...

# conv1 layer
model.add(Convolution2D(4, 3, 3, border_mode='same', activation='relu', input_shape=[img_size[0],img_size[1],3]))
model.add(Dropout(prob_drop_conv))

#2
model.add(Convolution2D(4, 3, 3, border_mode='same', activation='relu'))
#model.add(MaxPooling2D(pool_size=2, strides=(2,2), border_mode='same'))
model.add(Dropout(prob_drop_conv))

#3
model.add(Convolution2D(8, 3, 3, border_mode='same', activation='relu'))
model.add(Dropout(prob_drop_conv))

#4
model.add(Convolution2D(8, 3, 3, border_mode='same', activation='relu'))
model.add(Dropout(prob_drop_conv))

#5
model.add(Convolution2D(8, 3, 3, border_mode='same', activation='relu'))
#model.add(MaxPooling2D(pool_size=2, strides=(2,2), border_mode='same'))
model.add(Dropout(prob_drop_conv))

#6
model.add(Convolution2D(16, 3, 3, border_mode='same', activation='relu'))
model.add(Dropout(prob_drop_conv))

#7
model.add(Convolution2D(16, 3, 3, border_mode='same', activation='relu'))
#model.add(UpSampling2D())
model.add(Dropout(prob_drop_conv))

#8
model.add(Convolution2D(8, 3, 3, border_mode='same', activation='relu'))
model.add(Dropout(prob_drop_conv))

#9
model.add(Convolution2D(8, 3, 3, border_mode='same', activation='relu'))
model.add(Dropout(prob_drop_conv))

#10
model.add(Convolution2D(8, 3, 3, border_mode='same', activation='relu'))
#model.add(UpSampling2D())
model.add(Dropout(prob_drop_conv))

#11
model.add(Convolution2D(4, 3, 3, border_mode='same', activation='relu'))
model.add(Dropout(prob_drop_conv))

#12
model.add(Convolution2D(1, 3, 3, border_mode='same', activation='relu'))

model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

c:\users\t\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(4, (3, 3), padding="same", activation="relu", input_shape=[40, 128, ...)`
  
c:\users\t\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(4, (3, 3), padding="same", activation="relu")`
  if sys.path[0] == '':
c:\users\t\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:17: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(8, (3, 3), padding="same", activation="relu")`
c:\users\t\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:21: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(8, (3, 3), padding="same", activation="relu")`
c:\users\t\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:25: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(8, (3, 3), padding=

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_275 (Conv2D)          (None, 40, 128, 4)        112       
_________________________________________________________________
dropout_252 (Dropout)        (None, 40, 128, 4)        0         
_________________________________________________________________
conv2d_276 (Conv2D)          (None, 40, 128, 4)        148       
_________________________________________________________________
dropout_253 (Dropout)        (None, 40, 128, 4)        0         
_________________________________________________________________
conv2d_277 (Conv2D)          (None, 40, 128, 8)        296       
_________________________________________________________________
dropout_254 (Dropout)        (None, 40, 128, 8)        0         
_________________________________________________________________
conv2d_278 (Conv2D)          (None, 40, 128, 8)        584       
__________

In [57]:
model.fit_generator( train_generator, steps_per_epoch = 1000, epochs = 50)

Epoch 1/50
1000/1000 [==============================] - 31s 31ms/step - loss: nan - acc: 8.7500e-05
Epoch 2/50
1000/1000 [==============================] - 28s 28ms/step - loss: nan - acc: 8.3984e-05
Epoch 3/50
1000/1000 [==============================] - 27s 27ms/step - loss: nan - acc: 8.3984e-05
Epoch 4/50
1000/1000 [==============================] - 28s 28ms/step - loss: nan - acc: 8.3984e-05
Epoch 5/50
1000/1000 [==============================] - 29s 29ms/step - loss: nan - acc: 8.3984e-05
Epoch 6/50
1000/1000 [==============================] - 29s 29ms/step - loss: nan - acc: 8.3984e-050s - loss: nan - acc: 8.16
Epoch 7/50
1000/1000 [==============================] - 30s 30ms/step - loss: nan - acc: 8.3984e-05
Epoch 8/50
1000/1000 [==============================] - 30s 30ms/step - loss: nan - acc: 8.3984e-05
Epoch 9/50
1000/1000 [==============================] - 33s 33ms/step - loss: nan - acc: 8.3984e-05
Epoch 10/50
1000/1000 [==============================] - 32s 32ms/step - l

# Exc. 7.2 FCN mit Bottleneck

Implementiere jetzt die Variante mit schrittweisem Down- und Upsampling, wie in der Vorlesung besprochen. Nutze dafür ein bestehendes Netzwerk (z.B. VGG16, https://keras.io/applications/#vgg16), entferne die FC-Schichten am Ende, und füge dann die Upsampling-Schichten hinzu. Wie in der vorigen Vorlesung zu Transfer Learning beschrieben, kannst Du jetzt nur den zweiten Teil trainieren und die Gewichte des ersten Teils "einfrieren".

Stelle wie oben den Verlauf des Losses dar und wähle einige Beispielbilder aus dem Testset und zeige sie mit ihrer vorhergesagten Segmentierung an. (**BONUS**)